<a href="https://colab.research.google.com/github/vinayshanbhag/covid19/blob/master/covid_19_TX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests

In [17]:
pd.__version__

'1.0.3'

In [0]:
url_cases = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyCaseCountData.xlsx"
url_fatalities = "https://dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyFatalityCountData.xlsx"

In [19]:
data = requests.get(url_cases, verify=False).content

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
df_cases = pd.read_excel(data, sheet_name="COVID-19 Cases",skiprows=[0,1]).dropna(how='any')
df_cases.columns = ["2020-{}".format(c.split("\n")[-1]) if c.find("Cases \n")>=0 else "_".join(c.lower().split()) for c in df_cases.columns]

In [21]:
data = requests.get(url_fatalities, verify=False).content
df_fatalities = pd.read_excel(data, sheet_name="COVID-19 Fatalities",skiprows=[0,1]).dropna(how='any')
df_fatalities.columns = ["2020-{}".format(c.split("\n")[-1]) if c.find("Fatalities")>=0 else "_".join(c.lower().split()) for c in df_fatalities.columns]

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [0]:
df_cases = df_cases.melt(id_vars=["county_name", "population"], 
        var_name="date", 
        value_name="total_cases")

In [0]:
df_fatalities = df_fatalities.melt(id_vars=["county_name", "population"], 
        var_name="date", 
        value_name="total_fatalities")

In [0]:
df_cases["date"] = pd.to_datetime(df_cases.date)
df_cases["total_cases"] = pd.to_numeric(df_cases.total_cases)
df_fatalities["date"] = pd.to_datetime(df_fatalities.date)
df_fatalities["total_fatalities"] = pd.to_numeric(df_fatalities.total_fatalities)

In [0]:
df_tx = pd.merge(df_cases, df_fatalities, how="left",on=["county_name","population","date"])

In [0]:
df_tx.fillna(0,inplace=True)

In [0]:
t = df_tx.set_index(["county_name","date"]).groupby(level=0)['total_cases'].diff()
df_tx["new_cases"] = t.values
t = df_tx.set_index(["county_name","date"]).groupby(level=0)['total_fatalities'].diff()
df_tx["new_fatalities"] = t.values

In [0]:
top_counties = df_tx[df_tx.date==df_tx.date.max()].sort_values(by="total_cases",ascending=False)[:5].county_name.values
#counties = ["Travis","Williamson","Hays","Harris","Dallas"]

In [29]:
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt
import ipywidgets as widgets

widgets.interact_manual.opts['manual_name'] = 'Update chart'
@widgets.interact_manual(
    counties=widgets.SelectMultiple(
    options=list(df_tx.county_name.unique()),
    value=list(top_counties),
    #rows=10,
    description='Counties',
    disabled=False
)
)
def plot_growth(counties):
  plt.figure(figsize=(15,10))
  plt.xscale('log')
  plt.yscale('log')
  colors = list('rgbcmyk')
  for ix, county in enumerate(counties):
    X = df_tx[(df_tx.county_name==county)&(df_tx.total_cases>0)].total_cases
    Y = df_tx[(df_tx.county_name==county)&(df_tx.total_cases>0)].new_cases
    p = Polynomial.fit(X, Y, 2)
    plt.plot(*p.linspace(),f'{colors[ix]}:',X,Y,f'{colors[ix]}.')
    
  legend=[]
  for c in counties:
    legend.append(f"{c if c!='Total' else 'All'} county trend")
    legend.append(f"{c if c!='Total' else 'All'} county data")
  plt.legend(legend) 

  plt.xlabel("Total Cases")
  plt.ylabel("New Cases")
  plt.title("New Cases vs Total Cases");
  plt.minorticks_on()
  plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
  plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

interactive(children=(SelectMultiple(description='Counties', index=(254, 100, 56, 219, 226), options=('Anderso…

In [31]:
import ipywidgets as widgets

widgets.interact_manual.opts['manual_name'] = 'Update chart'
@widgets.interact_manual(
    county=widgets.Combobox(
    value='Williamson',
    placeholder='Select a county',
    options=list(df_tx.county_name.unique()),
    description='County:',
    ensure_option=True,
    disabled=False
)
)
def plot(county):
  X = df_tx[df_tx.county_name==county]["date"]
  Y = df_tx[df_tx.county_name==county]["total_cases"]
  Y2 = df_tx[df_tx.county_name==county]["total_fatalities"]
  plt.figure(figsize=(15,5))
  h = df_tx[df_tx.county_name==county]["total_cases"].max()/2
  k = (df_tx[(df_tx.county_name==county)].total_cases-h).abs().min()
  m = df_tx[(df_tx.county_name==county)&((h-df_tx.total_cases).abs()==k)]
  
  plt.plot(X,Y,'.-',X,Y2,'.-')
  plt.legend(["total_cases","total_fatalities"])
  plt.text(X.max(),Y.max(),Y.max())
  plt.text(X.max(),Y2.max(),Y2.max())
  plt.text(m.date,m.total_cases,f'({m.total_cases.values[0]:0.0f} cases, {(df_tx.date.max()-m.date).dt.days.values[0]} days ago)')
  plt.xlabel("Date")
  plt.ylabel("#")
  plt.title(f"{county+' County' if county!='Total' else 'All counties'}");
  plt.minorticks_on()
  plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
  plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

interactive(children=(Combobox(value='Williamson', description='County:', ensure_option=True, options=('Anders…